# Import Libraries

In [45]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, MaxPooling2D, Flatten,InputLayer Dense, Dropout, BatchNormalization,Conv2D
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
import keras_tuner as kt
import tensorflow_hub as hub

seed = 42
tf.random.set_seed(seed=seed)

train_path = 'data/kaggle_data/train_images'


# Image Data Processing

In [37]:

image_data_generator = ImageDataGenerator(rescale=1./255,
                                            rotation_range = 45,
                                            width_shift_range = 0.2,
                                            height_shift_range = 0.2,
                                            shear_range = 0.2,
                                            zoom_range = 0.2,
                                            horizontal_flip = True,
                                            vertical_flip = True,
                                            validation_split=0.2,
                                            fill_mode = 'nearest')

train_set = image_data_generator.flow_from_dataframe(data,
                         directory = train_path,
                         seed=seed,
                         target_size=(380,380),
                         subset='training',
                         x_col = 'image_id',
                         y_col = 'label_name',
                         class_mode = 'categorical',
                         shuffle = True,
                         batch_size = 32)

val_set = image_data_generator.flow_from_dataframe(data,
                         directory = train_path,
                         seed=seed,
                         target_size=(380,380),
                         subset='validation',
                         x_col = 'image_id',
                         y_col = 'label_name',
                         class_mode = 'categorical',
                         shuffle = False,
                         batch_size = 32)

Found 17118 validated image filenames belonging to 5 classes.
Found 4279 validated image filenames belonging to 5 classes.


# Modeling and Hyperparameter Tuning

In [43]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_filepath = 'model1.h5'
mc = ModelCheckpoint(checkpoint_filepath,
                    verbose=1,
                    save_weights_only=True,
                    monitor='val_loss',
                    mode='auto',
                    save_best_only=True
                    )

In [55]:
def build_model(hp):
  model = Sequential()

  model.add(InputLayer(input_shape = (380,380,3)))
  for i in range(hp.Int('conv_layers', min_value=0, max_value=3, step=1)):
    model.add(Conv2D(hp.Int(f'layer_{i}_filters', min_value=8,max_value=128,step=i*8),(3,3),activation='relu'))

  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dense(units=hp.Int('dense_layers',min_value=32,max_value=512,step=32),activation='relu'))

  if hp.Boolean("dropout"):
    model.add(Dropout(rate=0.25))
  model.add(Dense(5, 'softmax'))
  
  model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate',[1e-4, 1e-3, 1e-2])),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

  return model

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=2,
    executions_per_trial=2,
    overwrite=True,
    directory="model1_params",
    project_name="best_params")

tuner.search_space_summary()

Search space summary
Default search space size: 3
conv_layers (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 3, 'step': 1, 'sampling': None}
dense_layers (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
dropout (Boolean)
{'default': False, 'conditions': []}


In [ ]:
tuner.search(train_set, epochs=2, verbose=1, callbacks=[mc], validation_data=val_set)

In [57]:
best_model = tuner.get_best_hyperparameters()[0]

In [63]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(380, 380, 3))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 378, 378, 8)       224       
                                                                 
 conv2d_1 (Conv2D)           (None, 376, 376, 8)       584       
                                                                 
 conv2d_2 (Conv2D)           (None, 374, 374, 8)       584       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 187, 187, 8)      0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 279752)            0         
                                                                 
 dense (Dense)               (None, 160)               44760480  
                                                        

In [44]:
print('Summary:')
print('='*20)
for layer in model1.layers:
    print(f'{layer.name}: {str(layer.trainable)}')
print('='*20)

Summary:
conv2d_3: True
max_pooling2d_2: True
conv2d_4: True
max_pooling2d_3: True
conv2d_5: True
flatten_1: True
dense_2: True
dropout_1: True
dense_3: True
